In [ ]:
#https://www.kaggle.com/datasets/umairshahpirzada/traffic-net?resource=download
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from PIL import Image


ModuleNotFoundError: No module named 'tensorflow'

In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [ ]:
# Set the paths to the train and test directories
train_ds = "/kaggle/input/traffic-net/trafficnet_dataset_v1/train"
test_ds = "/kaggle/input/traffic-net/trafficnet_dataset_v1/test"

# Initialize ImageDataGenerator with rescaling for both training and testing
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15
)

test_datagen = ImageDataGenerator(rescale=1.0/255.)

# Create train and test generators
train_generator = train_datagen.flow_from_directory(
    train_ds,
    target_size=(224, 224),
    batch_size=32,
    shuffle=True,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_ds,
    target_size=(224, 224),
    batch_size=32,
    shuffle=False,
    class_mode='categorical'
)


In [ ]:
# Initialize the Sequential model
model = Sequential()

# Add layers to the model
model.add(Conv2D(64, (3, 3), padding="same", activation="relu", input_shape=(224, 224, 3)))
model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
model.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(256, (3, 3), padding="same", activation="relu"))
model.add(Conv2D(256, (3, 3), padding="same", activation="relu"))
model.add(Conv2D(256, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(512, (3, 3), padding="same", activation="relu"))
model.add(Conv2D(512, (3, 3), padding="same", activation="relu"))
model.add(Conv2D(512, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(4096, activation="relu"))
model.add(Dense(4096, activation="relu"))
model.add(Dense(4, activation="softmax"))

# Display the model summary
model.summary()


In [ ]:
# Compile the model with Adam optimizer and categorical crossentropy loss
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model using the train and test generators
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=20
)


In [ ]:
# Create a DataFrame for the training history
results = pd.DataFrame(history.history)
results.head()

# Plot model loss and accuracy
plt.figure(figsize=(15,6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['Train','Val'], loc= 'upper left')
plt.show()

plt.figure(figsize=(15,5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()


In [ ]:
# Get the true labels from the test set
y_test_1 = test_generator.classes

# Predict on the test set
y_pred_1 = model.predict(test_generator)
y_pred_1 = np.argmax(y_pred_1, axis=1)

# Evaluate the model on the test set
results = model.evaluate(test_generator)
print("Test loss:", results[0])
print("Test accuracy:", results[1])

# Display a classification report
from sklearn.metrics import classification_report
print(classification_report(y_test_1, y_pred_1))


In [ ]:
# Display predictions for a few test images
image_directory = '/kaggle/input/traffic-net/trafficnet_dataset_v1/test/dense_traffic'
img_size = 224

images = [] 
for filename in os.listdir(image_directory):
    path = os.path.join(image_directory, filename)
    img = Image.open(path)
    img = img.resize((img_size, img_size))
    images.append(img)

images = np.array([np.array(img) for img in images])
images = images / 255.0

predictions = model.predict(images)

# Select an image to display
img_index = 0

# Get predicted class label
class_label = np.argmax(predictions[img_index])

# Display image and predicted class label
plt.imshow(images[img_index])
plt.axis('off')
plt.title('Predicted class: ' + str(class_label))
plt.show()

# Print predictions for all images in the test set
for i in range(len(images)):
    predicted_class = np.argmax(predictions[i])
    class_probability = predictions[i, predicted_class]
    print(f'Predicted class for {i+1}.jpg : {test_generator.class_indices[predicted_class]}')
    print('Class probability:', class_probability)
